In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import os

from os.path import dirname



root_path = dirname(os.getcwd()) + "/HGNN_NA"

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


device(type='cuda', index=0)

In [2]:
dataset = "BPI_Challenge_2012_A"

In [3]:
tab_all = pd.read_csv(data_dir_processed+dataset+"_processed_all.csv")
print(tab_all.head())
list_activities = list(tab_all["Activity"].unique())

   org:resource lifecycle:transition           Activity       time:timestamp  \
0           112             COMPLETE        A_SUBMITTED  2011/09/30 22:38:44   
1           112             COMPLETE  A_PARTLYSUBMITTED  2011/09/30 22:38:44   
2           112             COMPLETE      A_PREACCEPTED  2011/09/30 22:39:37   
3         10862             COMPLETE         A_ACCEPTED  2011/10/01 09:42:43   
4         10862             COMPLETE        A_FINALIZED  2011/10/01 09:45:09   

         case:REG_DATE  CaseID  case:AMOUNT_REQ  
0  2011/10/01 00:38:44  173688            20000  
1  2011/10/01 00:38:44  173688            20000  
2  2011/10/01 00:38:44  173688            20000  
3  2011/10/01 00:38:44  173688            20000  
4  2011/10/01 00:38:44  173688            20000  


In [4]:
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [5]:
with open(data_dir_graphs + dataset + "_TRAIN.pkl", "rb") as f:
    X_train, Y_train = pickle.load(f)
with open(data_dir_graphs + dataset + "_VALID.pkl", "rb") as f:
    X_valid, Y_valid = pickle.load(f)
with open(data_dir_graphs + dataset + "_TEST.pkl", "rb") as f:
    X_test, Y_test = pickle.load(f)

In [24]:
from typing_extensions import Self
from torch_geometric.data import HeteroData
from torch_geometric.data import Dataset
from torch.utils.data import DataLoader
from torch_geometric.transforms import ToUndirected

transform = ToUndirected()

class Het_graph_data(Dataset):
    def __init__(self, prefix_graphs, labels) -> Self:
        self.X = prefix_graphs
        self.Y = labels

    # get the number of rows in the dataset
    def __len__(self):
        return len(self.Y)

    # get a row at a particular index in the dataset
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

    @staticmethod
    def collate(batch):
        # print(batch)
        data = [transform(item[0]) for item in batch]
        Y = [item[1] for item in batch]
        return [data, Y]

In [25]:
train_loader = DataLoader(
    Het_graph_data(X_train, Y_train),
    batch_size=1,
    shuffle=True,
    collate_fn=Het_graph_data.collate,
)


valid_loader = DataLoader(
    Het_graph_data(X_valid, Y_valid),
    batch_size=1,
    shuffle=False,
    collate_fn=Het_graph_data.collate,
)

test_loader = DataLoader(
    Het_graph_data(X_test, Y_test),
    batch_size=1,
    shuffle=False,
    collate_fn=Het_graph_data.collate,
)

In [26]:
# A Class to keep track of the metrics of the classification process
class ClassificationMetrics:

  # Constructor takes the number of classes, in our case 20
  def __init__(self, num_classes=20):
    self.num_classes = num_classes
    # Initialize a confusion matrix
    self.C = torch.zeros(num_classes, num_classes)

  # Update the confusion matrix with the new scores
  def add(self, yp, yt):
    # yp: 1D tensor with predictions
    # yt: 1D tensor with ground-truth targets
    yp = yp.to("cpu")
    yt = yt.to("cpu")
    with torch.no_grad(): # We require no computation graph
      self.C+=(yt*self.C.shape[1]+yp).bincount(minlength=self.C.numel()).view(self.C.shape).float()

  def clear(self):
    # We set the confusion matrix to zero
    self.C.zero_()

  # Computes the global accuracy
  def acc(self):
    return self.C.diag().sum().item()/self.C.sum()

  # Computes the class-averaged accuracy
  def mAcc(self):
    return (self.C.diag()/self.C.sum(-1)).mean().item()

  # Computers the class-averaged Intersection over Union
  def mIoU(self):
    return (self.C.diag()/(self.C.sum(0)+self.C.sum(1)-self.C.diag())).mean().item()

  # Returns the confusion matrix
  def confusion_matrix(self):
    return self.C

In [27]:
loaders = {"train": train_loader, "validation" : valid_loader, "test" : test_loader}

In [28]:
node_types, edge_types = X_train[0].metadata()

In [ ]:
X_train[0].x_dict

In [ ]:
node_types

In [ ]:
edge_types

In [29]:
# from models.models import HGNN
import datetime

from torch import nn
from tqdm.notebook import tqdm


In [46]:
from copy import deepcopy
from typing_extensions import Self
from torch_geometric.nn import SAGEConv, HeteroConv, GATConv # Linear, GCNConv
from torch.nn import ModuleList, Module, Sequential, Softmax, Dropout, Linear
from torch import mean, stack, sum, concat


class HGNN(Module):

    def __init__(self, hid, out, layers, node_types, nodes_relations) -> Self:  # type: ignore
        super().__init__()

        # List of convolutional layers
        self.convs = ModuleList()
        for _ in range(layers):
            conv = HeteroConv(
                    {relation:SAGEConv((-1, -1), hid) for relation in nodes_relations}
                    # {('Activity', 'followed_by', 'Activity') : SAGEConv((-1,-1), hid),
                    #  ("org:resource", "related_to", 'org:resource') : SAGEConv((-1,-1), hid),
                    #  ('time:timestamp', "related_to", "time:timestamp") : SAGEConv((-1,-1), hid)}
                    ,
                    aggr="sum",
                )
            self.convs.append(conv)
            
        #print(nodes_relations)
        # Take each node hid representation and apply a linear layer
        self.linear_nodes = Sequential(Linear(hid, hid), Dropout(p=0.7), Linear(hid, int(hid / 2)))

        # Return the softmax with the class probabilities
        self.fc = Sequential(Linear(int(hid/2)*(len(node_types)), out), Softmax(dim=0))

    def forward(self, x_dict, edge_index_dict):
        
        # x_dict = x_dict[0]
        # edge_index_dict = edge_index_dict[0]
        # Convolutional layers
        # print(x_dict)
        # x_dict_copy = deepcopy(x_dict)
        # print(edge_index_dict)
        for conv in self.convs:
            #print("HEYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
            #print(x_dict)
            #print(conv)
            x_dict = conv(x_dict, edge_index_dict)
            #print(x_dict)
            #print("HEYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
            x_dict = {key: x.relu() for key, x in x_dict.items()}

        # print(x_dict)
        # Node features of each node in the graph
        nodes_features = [
            self.linear_nodes(x_dict[key]).relu() for key in x_dict.keys()
        ]
        # print(nodes_features)
        # Global mean of each node type
        for i in range(len(nodes_features)):
            nodes_features[i] = mean(nodes_features[i], dim=0)

        # print(nodes_features)
        # print(concat(nodes_features))
        # Global mean pooling
        #nodes_features = mean(stack(nodes_features), dim=0)
        nodes_features = concat(nodes_features)
        nodes_features = self.fc(nodes_features)

        return nodes_features  # {key : self.linear(x_dict[key]) for key in x_dict.keys()}, nodes_features


org:resource, related_to, org:resource)={
    edge_attr=[1, 1],
    edge_index=[2, 1],
  },
  (Activity, followed_by, Activity)={
    edge_attr=[1, 1],
    edge_index=[2, 1],
  },
  (time:timestamp, related_to, time:timestamp)

In [47]:
model = HGNN(hid=128, out=len(list_activities), layers=4, node_types=node_types, nodes_relations=edge_types)
model

HGNN(
  (convs): ModuleList(
    (0-1): 2 x HeteroConv(num_relations=8)
  )
  (linear_nodes): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): Dropout(p=0.3, inplace=False)
    (2): Linear(in_features=32, out_features=16, bias=True)
  )
  (fc): Sequential(
    (0): Linear(in_features=96, out_features=10, bias=True)
    (1): Softmax(dim=0)
  )
)

In [ ]:
X_train[0].x_dict.items()

In [ ]:
X_train[0].edge_items()

In [ ]:
from torch_geometric.transforms import ToUndirected

X_train[0] = ToUndirected()(X_train[0])

In [ ]:
X_train[0].x_dict

In [ ]:
X_train[0].edge_items()

In [ ]:
X_train[0].is_directed()

In [ ]:
with torch.no_grad():
    print(model(X_train[0].x_dict,X_train[0].edge_index_dict))
    print(Y_train[0])

In [ ]:
k_t = {}
for i,(x,y) in enumerate(loaders['test']):
    _,pr = torch.max(y[0], dim=0)
    try:
        k_t[pr.item()] += 1
    except KeyError:
        k_t[pr.item()] = 1
k_t

In [ ]:
k_t = []
for i,(x,y) in enumerate(loaders['validation']):
    k_t.append("__".join(list(x[0].x_dict.keys())))
    

In [ ]:
k_t[:5]

In [ ]:
k_t_s = set(k_t)
k_t_s

In [ ]:
X_train[2]

In [ ]:
from torch import tensor
m  =  ClassificationMetrics(num_classes=5)
m.add(tensor([1]),tensor([2]))
print(m.confusion_matrix())

In [48]:
from torch import int32


num_epochs = 3
best_accuracy = 0
early_stop_patience = 10
lr_value = 0.001

best_model = None

num_runs = 1
running_time = []

metric_tracker = ClassificationMetrics(num_classes=len(list_activities))

for run in range(num_runs):
    
    start = datetime.datetime.now()
    print("Run: {}".format(run + 1))
    
    
    
    model = model.to(device)
    #print("compiling...")
    #model = torch.compile(model, dynamic=True)
    #print("Compiled model")
    
    criterion = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr_value) # momentum=0.9, weight_decay=1e-1)

    
    not_improved_count = 0
    
    for epoch in range(num_epochs):
        print(
            "\n-- EPOCH {}/{} -------------------------\n".format(epoch + 1, num_epochs)
        )
        torch.cuda.empty_cache()
        count_train = [0 for _ in range(len(list_activities))]
        count_true_train = [0 for _ in range(len(list_activities))]
        count_val = [0 for _ in range(len(list_activities))]
        count_true_val = [0 for _ in range(len(list_activities))]
        for state in ["train", "validation"]:
            if state == "train":
                model.train()
                metric_tracker.clear()
                # true_label = []
                # predictions = []
                # scores = []

            else:
                
                # print(metric_tracker.confusion_matrix())
                #print(count_train)
                #print(count_true_train)
                
                print("\tTRAIN | acc: {:.4f} | mAcc: {:.4f} | mIoU: {:.4f}".format(metric_tracker.acc(),
                                                                                   #metric_tracker.mAcc(),
                                                                                   #metric_tracker.mIoU()
                                                                                   0,0))
                
                # y_true = np.concatenate(true_label)
                # y_pred = np.concatenate(predictions)
                # scores = np.concatenate(scores)
                
                # print("TRAIN")
                # print(conta(y_true, 0), conta(y_true, 1))
                # print(conta(y_pred, 0), conta(y_pred, 1))
                
                # print_stats(y_pred, y_true, scores)
                metric_tracker.clear()
                model.eval()
                # true_label = []
                # predictions = []
                # scores = []
            
            running_loss = 0.0
            running_corrects = 0

            for i,(x,y) in tqdm(enumerate(loaders[state])):
                
                if i % 1000 == 1:
                    print(metric_tracker.acc())
                    # print(running_loss/i)
                    print(metric_tracker.confusion_matrix())
                    #print(count_train)
                    #print(count_true_train)
                    print("//"*14)
                # print("X")
                # print("//"*50)
                # print(x)
                # print("//"*50)
                # print(y)
                x = x[0].to(device)
                # print("//"*50)
                #print(x)
                #print(x.edge_index_dict)
                #print(x.x_dict)
                # print("//"*50)
                
                y = y[0].to(device)
                
                #x = [[sub_item.to(device=device) for sub_item in item] for item in x]


                #y = torch.tensor([torch.max(yi,0)[1] for yi in y])

                #y = y.to(device)
                
                outputs = model(x.x_dict, x.edge_index_dict)
                
                # print(outputs)
                # print("//"*30)
                
                
                # outputs = outputs.to(device)
                
                
                
                loss = criterion(outputs, y)

                if state == "train":
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                _, preds = torch.max(outputs, 0)
                
                # print(y)
                _, true_preds = torch.max(y, 0)                  
                # print(true_preds)
                preds = preds.to(device)
                true_preds = true_preds.to(device)
                if state == "train": # For now batch is set to one
                    #for i in preds:
                    count_train[preds] += 1
                    count_true_train[true_preds] += 1
                    
                else:
                    #for i in preds:
                    count_val[preds] += 1      
                    count_true_val[true_preds] += 1      
                      
                      
                running_loss += loss.item()
                
                # print(outputs, y, sep="\n")
                # print(preds, true_preds, sep="\n")
                metric_tracker.add(torch.tensor([preds.to(int32)]), torch.tensor([true_preds.to(int32)]))
                # print(metric_tracker.confusion_matrix())
                
                
        print(count_val)
        print("\tEVAL  | acc: {:.4f} | mAcc: {:.4f} | mIoU: {:.4f}\n".format(metric_tracker.acc(),
                                                                             # metric_tracker.mAcc(),
                                                                             # metric_tracker.mIoU()
                                                                             # )
                                                                            0,0)   )     
                   

        if epoch == 0:# HERE WE KEEP BEST AUC VALUE
            best_accuracy = metric_tracker.acc()
            
        else:
            if metric_tracker.acc() > best_accuracy:
                print("SAVING MODEL..............\n")
                best_accuracy = metric_tracker.acc()
                not_improved_count = 0
            else:
                not_improved_count += 1

        if not_improved_count == early_stop_patience:
            print(
                "Validation performance didn't improve for {} epochs. "
                "Training stops.".format(early_stop_patience)
            )
            break

    running_time.append((datetime.datetime.now() - start).total_seconds())

Run: 1

-- EPOCH 1/3 -------------------------



0it [00:00, ?it/s]

tensor(0.)
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
////////////////////////////
tensor(0.0709)
tensor([[ 0., 19.,  0.,  1.,  0., 29.,  1., 20.,  1., 68.],
        [ 0.,  1.,  0.,  0.,  0., 16.,  0.,  3.,  3., 38.],
        [ 0.,  4.,  0.,  1.,  0., 14.,  0.,  5.,  1., 34.],
        [ 0.,  5.,  0.,  0.,  0., 30.,  1., 14.,  5., 45.],
        [ 0., 12.,  6.,  0.,  3., 21., 19., 70., 13., 93.],
        [ 0.,  7.,  0.,  0.,  0., 38.,  0., 12.,  3., 68.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  1., 

KeyboardInterrupt: 